## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



toolz-0.12.1         | 51 KB     | ##################################### | 100% 
conda-4.14.0         | 1010 KB   | ##################################### | 100% 
Preparing transaction: done
Ver

## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(1):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:07<00:00, 14.76it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_0"])
layer_hs_array.shape

(10000, 267264)

## Layer 0 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

# dim_reduced_vecs = layer_hs_array

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 7894)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 7894D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 0.04 s
Outer iteration 0 / 10
  Iteration 19 (0.83 s, search 0.56 s): objective=2944.9 imbalance=1.183 nsplit=0        
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (1.66 s, search 1.13 s): objective=2980.92 imbalance=1.417 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (2.48 s, search 1.69 s): objective=2938.47 imbalance=1.332 nsplit=0       
Outer iteration 3 / 10
  Iteration 19 (3.31 s, search 2.25 s): objective=2964.66 imbalance=1.206 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (4.13 s, search 2.82 s): objective=2917.98 imbalance=1.266 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (4.95 s, search 3.37 s): objective=2979.26 imbalance=1.158 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (5.80 s, search 3.95 s): objective=2954.74 imbalance=1.146 nsplit=0       
Outer iteration 7 / 10
  Iter

2980.919921875

In [10]:
kmeans.centroids #cluster centers

array([[ 0.01780193,  0.00530282,  0.00193194, ...,  0.00733415,
        -0.01534147, -0.01299665],
       [ 0.00251477,  0.01532933,  0.00086739, ..., -0.0168123 ,
        -0.01393857, -0.00398347],
       [ 0.00401511,  0.002     ,  0.00808934, ..., -0.00815284,
        -0.00518691, -0.00325297],
       ...,
       [ 0.02030369,  0.02499921, -0.00698511, ..., -0.00193098,
        -0.00922605, -0.00795133],
       [ 0.02338583,  0.00987874,  0.0127636 , ..., -0.01273464,
        -0.00274767, -0.007074  ],
       [-0.00159311,  0.01607657, -0.00530805, ..., -0.01036403,
        -0.00517196, -0.00640689]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.0
0.99999976
1.0
0.9999996
1.0
1.0000001
1.0000001
1.0000001
0.9999999
1.0000001


In [12]:
kmeans.obj #inertia at each iteration

array([1041.63989258, 2790.0456543 , 2878.58447266, 2909.56054688,
       2923.90014648, 2931.49487305, 2936.73193359, 2938.67822266,
       2939.73583984, 2940.6953125 , 2942.02978516, 2943.99731445,
       2944.62792969, 2944.79272461, 2944.87670898, 2944.90209961,
       2944.90209961, 2944.90209961, 2944.90209961, 2944.90209961,
       1063.11108398, 2807.36303711, 2898.03466797, 2934.63525391,
       2952.27539062, 2967.33569336, 2971.78637695, 2975.09594727,
       2977.96557617, 2979.83642578, 2980.60083008, 2980.82641602,
       2980.88671875, 2980.91503906, 2980.91699219, 2980.91992188,
       2980.91992188, 2980.91992188, 2980.91992188, 2980.91992188])

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

9    2711
1    1488
5    1003
8     922
7     917
6     902
2     651
0     555
3     445
4     406
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

9    2711
1    1488
5    1003
8     922
7     917
6     902
2     651
0     555
3     445
4     406
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[8, 6, 0, 9, 2, 5, 6, 8, 4, 7]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[3, 6, 5, 3, 9, 6, 2, 10, 8, 1]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

# max_labels = [label for label in max_label_per_centroid for _ in range(1000)] #DOESN'T MAKE SENSE - INDICES ARE CENTROIDS, NOT LABELS - DOESN'T MATCH DATA

In [21]:
np.array(max_centroids)

array([8, 8, 8, ..., 7, 7, 7])

In [22]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [23]:
label_to_centroid

{1: 8, 2: 6, 3: 0, 4: 9, 5: 2, 6: 5, 7: 6, 8: 8, 9: 4, 10: 7}

In [24]:
centroid_to_label

{0: 3, 1: 6, 2: 5, 3: 3, 4: 9, 5: 6, 6: 2, 7: 10, 8: 8, 9: 1}

In [25]:
print(pd.Series(prompt_ids[centroid_labels[0]]).value_counts())

print(pd.Series(prompt_ids[centroid_labels[3]]).value_counts())

3    555
dtype: int64
3    445
dtype: int64


In [26]:
print(pd.Series(prompt_ids[centroid_labels[2]]).value_counts())

print(pd.Series(prompt_ids[centroid_labels[9]]).value_counts())

5    650
4      1
dtype: int64
1     368
4     356
7     334
2     328
9     326
5     307
8     306
6     304
10     82
dtype: int64


In [27]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 6,  6,  6, ..., 10, 10, 10])

In [28]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.368
Prompt 2 Accuracy:  0.38
Prompt 3 Accuracy:  1.0
Prompt 4 Accuracy:  0.0
Prompt 5 Accuracy:  0.65
Prompt 6 Accuracy:  0.664
Prompt 7 Accuracy:  0.0
Prompt 8 Accuracy:  0.492
Prompt 9 Accuracy:  0.401
Prompt 10 Accuracy:  0.917


In [29]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.4872


In [30]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.42
Prompt 2 Accuracy:  0.38
Prompt 3 Accuracy:  0.555
Prompt 4 Accuracy:  0.356
Prompt 5 Accuracy:  0.65
Prompt 6 Accuracy:  0.436
Prompt 7 Accuracy:  0.363
Prompt 8 Accuracy:  0.492
Prompt 9 Accuracy:  0.401
Prompt 10 Accuracy:  0.917


In [31]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.497


## Measure Conversion Accuracy

In [32]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [33]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [34]:
np.mean(max_centroids_to_labels == prompt_ids) #<100% - 7/10 label to centroid to label mappings coincide

0.7